# BCom Offshore Recommendation Model Development

## Train and Export Recommendation Model for Link Performance Optimization

This notebook develops a recommendation model trained on site grades to generate actionable insights
for improving link performance across the BCom Offshore network.

## 1. Import Required Libraries and Setup

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import logging
import warnings
warnings.filterwarnings('ignore')

# ML libraries
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import (
    classification_report, confusion_matrix, roc_auc_score, 
    precision_recall_curve, f1_score, accuracy_score
)
import joblib

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

# Custom modules
from app.services.data_loader import get_data_loader
from app.services.model_management import ModelManager

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

print("✓ All libraries imported successfully")

## 2. Load and Explore Data

In [ ]:
# Initialize DataLoader
data_loader = get_data_loader('data')

# Get customer data for ML
customer_id = 4  # Orange MALI
print(f"Loading data for Customer ID: {customer_id}...")

# Get customer summary
customer_summary = data_loader.get_customer_summary(customer_id)
print(f"\nCustomer Summary:")
print(f"  Networks: {customer_summary.get('network_count', 'N/A')}")
print(f"  Sites: {customer_summary.get('site_count', 'N/A')}")
print(f"  Links: {customer_summary.get('link_count', 'N/A')}")
print(f"  Devices: {customer_summary.get('device_count', 'N/A')}")

# Export ML data
ml_data = data_loader.export_customer_data_for_ml(customer_id)
print(f"\nLoaded {len(ml_data)} records for training")
print(f"\nData shape: {ml_data.shape}")
print(f"Columns: {ml_data.columns.tolist()}")
print(f"\nFirst few rows:")
display(ml_data.head())

## 3. Data Preprocessing and Feature Engineering

In [ ]:
# Create working copy
df = ml_data.copy()

# Handle missing values
print(f"Missing values before handling:")
print(df.isnull().sum())

# Fill missing numeric columns with median
numeric_cols = df.select_dtypes(include=[np.number]).columns
for col in numeric_cols:
    if df[col].isnull().any():
        df[col].fillna(df[col].median(), inplace=True)

# Drop rows with missing target variable
df = df.dropna(subset=['grade'])

print(f"\nMissing values after handling: {df.isnull().sum().sum()}")
print(f"Data shape: {df.shape}")

# Display statistics
print(f"\nGrade distribution:")
print(df['grade'].describe())
print(f"\nGrade value counts:")
print(df['grade'].value_counts().sort_index())

## 4. Feature Engineering for Recommendations

In [ ]:
# Create target variable: links needing recommendations (grade < 7 or degrading)
df['needs_recommendation'] = (df['grade'] < 7.0).astype(int)

print(f"Recommendation target distribution:")
print(df['needs_recommendation'].value_counts())
print(f"\nClass balance: {df['needs_recommendation'].value_counts(normalize=True).round(3)}")

# Feature engineering
feature_engineering_cols = []

# 1. Grade quality score (0-1 scale)
df['grade_quality'] = df['grade'] / 10.0
feature_engineering_cols.append('grade_quality')

# 2. Performance indicators
performance_cols = ['availability', 'performance', 'congestion', 'latency']
available_perf_cols = [col for col in performance_cols if col in df.columns]
feature_engineering_cols.extend(available_perf_cols)

# 3. Stability indicators
stability_cols = ['ib_instability', 'ob_instability']
available_stab_cols = [col for col in stability_cols if col in df.columns]
feature_engineering_cols.extend(available_stab_cols)

# 4. Degradation factors
degradation_cols = ['ib_degradation', 'ob_degradation']
available_degrad_cols = [col for col in degradation_cols if col in df.columns]
feature_engineering_cols.extend(available_degrad_cols)

# 5. Uptime score
if 'up_time' in df.columns:
    df['uptime_score'] = df['up_time'] / 100.0
    feature_engineering_cols.append('uptime_score')

# Remove duplicates and clean
feature_engineering_cols = list(set(feature_engineering_cols))
feature_engineering_cols = [col for col in feature_engineering_cols if col in df.columns]

print(f"\nFeatures selected for model ({len(feature_engineering_cols)}):")
print(feature_engineering_cols)

# Verify all features are numeric
for col in feature_engineering_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

print(f"\nFeature statistics:")
display(df[feature_engineering_cols].describe())

## 5. Train Recommendation Model

In [ ]:
# Prepare X and y
X = df[feature_engineering_cols].copy()
y = df['needs_recommendation'].copy()

# Remove any remaining NaN values
mask = X.isnull().any(axis=1) | y.isnull()
X = X[~mask]
y = y[~mask]

print(f"Training set size: {len(X)}")
print(f"Features: {X.shape[1]}")
print(f"Target distribution: {y.value_counts().to_dict()}")

# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled = pd.DataFrame(X_scaled, columns=X.columns)

print(f"\nFeatures scaled successfully")
print(f"Scaled features mean: {X_scaled.mean().mean():.6f}")
print(f"Scaled features std: {X_scaled.std().mean():.6f}")

In [ ]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Training set: {X_train.shape[0]} samples")
print(f"Test set: {X_test.shape[0]} samples")
print(f"\nTraining set class distribution:")
print(pd.Series(y_train).value_counts())
print(f"\nTest set class distribution:")
print(pd.Series(y_test).value_counts())

In [ ]:
# Train Gradient Boosting model
print("Training Gradient Boosting Classifier...")

gb_model = GradientBoostingClassifier(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=5,
    min_samples_split=10,
    min_samples_leaf=5,
    subsample=0.8,
    random_state=42,
    verbose=0
)

gb_model.fit(X_train, y_train)

# Training metrics
train_pred = gb_model.predict(X_train)
train_pred_proba = gb_model.predict_proba(X_train)[:, 1]

print(f"\n✓ Model trained successfully")
print(f"Training Accuracy: {accuracy_score(y_train, train_pred):.4f}")
print(f"Training AUC-ROC: {roc_auc_score(y_train, train_pred_proba):.4f}")

## 6. Model Evaluation

In [ ]:
# Test predictions
y_pred = gb_model.predict(X_test)
y_pred_proba = gb_model.predict_proba(X_test)[:, 1]

# Evaluation metrics
test_accuracy = accuracy_score(y_test, y_pred)
test_auc = roc_auc_score(y_test, y_pred_proba)
test_f1 = f1_score(y_test, y_pred)

print("═" * 50)
print("MODEL EVALUATION RESULTS")
print("═" * 50)
print(f"Test Accuracy:  {test_accuracy:.4f}")
print(f"Test AUC-ROC:   {test_auc:.4f}")
print(f"Test F1-Score:  {test_f1:.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=['No Recommendation', 'Recommend Action']))

In [ ]:
# Confusion matrix visualization
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[0], cbar=False)
axes[0].set_title('Confusion Matrix', fontsize=12, fontweight='bold')
axes[0].set_ylabel('True Label')
axes[0].set_xlabel('Predicted Label')
axes[0].set_xticklabels(['No Recommend', 'Recommend'])
axes[0].set_yticklabels(['No Recommend', 'Recommend'])

# Precision-Recall Curve
precision, recall, _ = precision_recall_curve(y_test, y_pred_proba)
axes[1].plot(recall, precision, marker='o', linewidth=2, label='Model')
axes[1].set_xlabel('Recall')
axes[1].set_ylabel('Precision')
axes[1].set_title('Precision-Recall Curve', fontsize=12, fontweight='bold')
axes[1].grid(True, alpha=0.3)
axes[1].legend()

plt.tight_layout()
plt.show()

print("✓ Evaluation plots created")

## 7. Feature Importance Analysis

In [ ]:
# Feature importance
feature_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': gb_model.feature_importances_
}).sort_values('importance', ascending=False)

print("\nFeature Importance (Top 10):")
print(feature_importance.head(10).to_string())

# Visualization
fig, ax = plt.subplots(figsize=(10, 6))
top_features = feature_importance.head(10)
ax.barh(range(len(top_features)), top_features['importance'])
ax.set_yticks(range(len(top_features)))
ax.set_yticklabels(top_features['feature'])
ax.set_xlabel('Importance Score')
ax.set_title('Top 10 Most Important Features for Recommendations', fontweight='bold')
ax.invert_yaxis()
plt.tight_layout()
plt.show()

print("\n✓ Feature importance visualized")

## 8. Cross-Validation Analysis

In [ ]:
# Cross-validation
cv_scores = cross_val_score(gb_model, X_scaled, y, cv=5, scoring='roc_auc')

print("\n5-Fold Cross-Validation Results:")
print(f"Scores: {cv_scores.round(4)}")
print(f"Mean: {cv_scores.mean():.4f}")
print(f"Std Dev: {cv_scores.std():.4f}")

# Visualization
fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(range(1, 6), cv_scores, marker='o', linewidth=2, markersize=8)
ax.axhline(y=cv_scores.mean(), color='r', linestyle='--', label=f'Mean: {cv_scores.mean():.4f}')
ax.fill_between(
    range(1, 6),
    cv_scores.mean() - cv_scores.std(),
    cv_scores.mean() + cv_scores.std(),
    alpha=0.2
)
ax.set_xlabel('Fold')
ax.set_ylabel('AUC-ROC Score')
ax.set_title('Cross-Validation Results', fontweight='bold')
ax.set_xticks(range(1, 6))
ax.grid(True, alpha=0.3)
ax.legend()
plt.tight_layout()
plt.show()

## 9. Save and Export Model

In [ ]:
# Initialize model manager
model_manager = ModelManager()

# Prepare model metadata
metadata = {
    'model_type': 'GradientBoostingClassifier',
    'task': 'recommendation_engine',
    'training_samples': len(X_train),
    'test_samples': len(X_test),
    'features': X.columns.tolist(),
    'feature_count': len(X.columns),
    'scaler_type': 'StandardScaler',
    'metrics': {
        'test_accuracy': float(test_accuracy),
        'test_auc_roc': float(test_auc),
        'test_f1_score': float(test_f1)
    },
    'hyperparameters': {
        'n_estimators': 100,
        'learning_rate': 0.1,
        'max_depth': 5,
        'subsample': 0.8
    },
    'training_timestamp': datetime.now().isoformat(),
    'data_source': 'BCom Offshore - Orange MALI Customer',
    'description': 'Recommendation engine for identifying links needing performance optimization'
}

# Save model and scaler
print("Saving model and preprocessing artifacts...")
model_manager.save_model(
    model=gb_model,
    model_name='recommendation_engine',
    version='1.0.0',
    metadata=metadata
)

# Save scaler
scaler_path = model_manager.model_dir / 'recommendation_engine' / '1.0.0' / 'scaler.pkl'
joblib.dump(scaler, scaler_path)

print(f"✓ Model saved to: {model_manager.model_dir}")
print(f"✓ Version: 1.0.0")
print(f"✓ Test AUC-ROC: {test_auc:.4f}")

## 10. Model Inference Example

In [ ]:
# Load saved model for inference
from app.services.model_loaders import RecommendationModelLoader

print("Testing inference with saved model...")

# Get a few test samples
test_sample_indices = np.where(y_test.values == 1)[0][:3]
test_samples = X_test.iloc[test_sample_indices]

# Make predictions
predictions = gb_model.predict(test_samples)
probabilities = gb_model.predict_proba(test_samples)

print("\nSample Recommendations:")
print("="*60)
for i, (idx, row) in enumerate(test_samples.iterrows()):
    prob_no_rec = probabilities[i][0]
    prob_rec = probabilities[i][1]
    
    print(f"\nSample {i+1}:")
    print(f"  Prediction: {'RECOMMEND' if predictions[i] == 1 else 'NO ACTION'}")
    print(f"  Confidence: {max(prob_no_rec, prob_rec):.2%}")
    print(f"  Prob(No Recommendation): {prob_no_rec:.4f}")
    print(f"  Prob(Recommendation): {prob_rec:.4f}")

print("\n✓ Inference test completed")

## 11. Model Performance Summary

In [ ]:
# Summary report
summary_report = f"""
╔════════════════════════════════════════════════════════════════╗
║         RECOMMENDATION MODEL TRAINING SUMMARY                  ║
╚════════════════════════════════════════════════════════════════╝

📊 DATASET INFORMATION:
   • Total Samples: {len(X)}
   • Training Samples: {len(X_train)}
   • Test Samples: {len(X_test)}
   • Features: {len(X.columns)}
   • Class Balance: {y.value_counts().to_dict()}

🤖 MODEL CONFIGURATION:
   • Algorithm: Gradient Boosting Classifier
   • Estimators: 100
   • Learning Rate: 0.1
   • Max Depth: 5
   • Scaler: StandardScaler

📈 TRAINING PERFORMANCE:
   • Training Accuracy: {accuracy_score(y_train, train_pred):.4f}
   • Training AUC-ROC: {roc_auc_score(y_train, train_pred_proba):.4f}

✅ TEST PERFORMANCE:
   • Test Accuracy: {test_accuracy:.4f}
   • Test AUC-ROC: {test_auc:.4f}
   • Test F1-Score: {test_f1:.4f}
   • CV Mean (5-fold): {cv_scores.mean():.4f} ± {cv_scores.std():.4f}

🎯 TOP 3 MOST IMPORTANT FEATURES:
   1. {feature_importance.iloc[0]['feature']}: {feature_importance.iloc[0]['importance']:.4f}
   2. {feature_importance.iloc[1]['feature']}: {feature_importance.iloc[1]['importance']:.4f}
   3. {feature_importance.iloc[2]['feature']}: {feature_importance.iloc[2]['importance']:.4f}

💾 MODEL EXPORT:
   • Name: recommendation_engine
   • Version: 1.0.0
   • Location: ml_models/recommendations/1.0.0/
   • Files: model.pkl, scaler.pkl, metadata.json

🚀 READY FOR PRODUCTION
"""

print(summary_report)

## 12. Next Steps

In [ ]:
next_steps = """
📋 NEXT STEPS:

1. INTEGRATE WITH API ENDPOINTS:
   ✓ Anomaly detection endpoint: POST /api/v1/anomalies/bcom/device/{device_id}/detect
   ✓ Link recommendations: POST /api/v1/recommendations/bcom/link/{link_id}/recommend
   ✓ Network health report: POST /api/v1/recommendations/bcom/network/{network_id}/health-report
   ✓ Customer improvement plan: GET /api/v1/recommendations/bcom/customer/{customer_id}/improvement-plan

2. DATABASE INTEGRATION:
   • Run migrations to create tables
   • Load data with ingestion pipeline
   • Test model inference with live data

3. MONITORING & OPTIMIZATION:
   • Track model performance in production
   • Monitor recommendation accuracy
   • Plan for periodic model retraining

4. ADVANCED FEATURES:
   • A/B testing for different models
   • Real-time model updates
   • Custom recommendation rules
"""

print(next_steps)